## Exercises

Within your `codeup-data-science` directory, create a new repo named `time-series-exercises`. This will be where you do your work for this module. Create a repository on GitHub with the same name, and link your local repository to GitHub.

Save this work in your `time-series-exercises` repo. Then add, commit, and push your changes.

The end result of this exercise should be a file named `acquire.py`.

1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named `items` that has all of the data for items.

1. Do the same thing, but for `stores` (https://python.zgulde.net/api/v1/stores)

1. Extract the data for `sales` (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

1. Save the data in your files to local csv files so that it will be faster to access in the future.

1. Combine the data from your three separate dataframes into one large dataframe. 

1. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

1. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the `acquire.py` file and be able to re-run the functions and get the same data.

In [1]:
import pandas as pd
import requests
import os

In [2]:
base_url = 'https://python.zgulde.net'
print(requests.get(base_url).text)

{"api":"/api/v1","help":"/documentation"}



In [3]:
response = requests.get(base_url + '/documentation')
response.json()

{'payload': '\nThe API accepts GET requests for all endpoints, where endpoints are prefixed\nwith\n\n    /api/{version}\n\nWhere version is "v1"\n\nValid endpoints:\n\n- /stores[/{store_id}]\n- /items[/{item_id}]\n- /sales[/{sale_id}]\n\nAll endpoints accept a `page` parameter that can be used to navigate through\nthe results.\n',
 'status': 'ok'}

In [4]:
response.json().keys()

dict_keys(['payload', 'status'])

In [5]:
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



### Creating the 'items' DataFrame

In [41]:
response = requests.get(base_url + '/api/v1/items')
data = response.json()

In [42]:
items_df = pd.DataFrame(data['payload']['items'])
items_df.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [43]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()
df = pd.concat([items_df, pd.DataFrame(data['payload']['items'])]).reset_index()
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()
items_df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index()

In [44]:
items_df.shape

(50, 8)

In [45]:
items_df

,level_0,index,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,0,0.0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,1.0,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,2,2.0,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,3,3.0,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,4,4.0,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036
5,5,5.0,Sally Hansen,6,Sally Hansen Nail Color Magnetic 903 Silver El...,6.93,74170388732,74170388732
6,6,6.0,Twinings Of London,7,Twinings Of London Classics Lady Grey Tea - 20 Ct,9.64,70177154004,70177154004
7,7,7.0,Lea & Perrins,8,Lea & Perrins Marinade In-a-bag Cracked Pepper...,1.68,51600080015,51600080015
8,8,8.0,Van De Kamps,9,Van De Kamps Fillets Beer Battered - 10 Ct,1.79,19600923015,19600923015
9,9,9.0,Ahold,10,Ahold Cocoa Almonds,3.17,688267141676,688267141676


### Creating the 'stores' DataFrame

In [6]:
response = requests.get(base_url + '/api/v1/stores')

In [7]:
data =  response.json()

In [8]:
data

{'payload': {'max_page': 1,
  'next_page': None,
  'page': 1,
  'previous_page': None,
  'stores': [{'store_address': '12125 Alamo Ranch Pkwy',
    'store_city': 'San Antonio',
    'store_id': 1,
    'store_state': 'TX',
    'store_zipcode': '78253'},
   {'store_address': '9255 FM 471 West',
    'store_city': 'San Antonio',
    'store_id': 2,
    'store_state': 'TX',
    'store_zipcode': '78251'},
   {'store_address': '2118 Fredericksburg Rdj',
    'store_city': 'San Antonio',
    'store_id': 3,
    'store_state': 'TX',
    'store_zipcode': '78201'},
   {'store_address': '516 S Flores St',
    'store_city': 'San Antonio',
    'store_id': 4,
    'store_state': 'TX',
    'store_zipcode': '78204'},
   {'store_address': '1520 Austin Hwy',
    'store_city': 'San Antonio',
    'store_id': 5,
    'store_state': 'TX',
    'store_zipcode': '78218'},
   {'store_address': '1015 S WW White Rd',
    'store_city': 'San Antonio',
    'store_id': 6,
    'store_state': 'TX',
    'store_zipcode': '78220

In [9]:
data['payload']

{'max_page': 1,
 'next_page': None,
 'page': 1,
 'previous_page': None,
 'stores': [{'store_address': '12125 Alamo Ranch Pkwy',
   'store_city': 'San Antonio',
   'store_id': 1,
   'store_state': 'TX',
   'store_zipcode': '78253'},
  {'store_address': '9255 FM 471 West',
   'store_city': 'San Antonio',
   'store_id': 2,
   'store_state': 'TX',
   'store_zipcode': '78251'},
  {'store_address': '2118 Fredericksburg Rdj',
   'store_city': 'San Antonio',
   'store_id': 3,
   'store_state': 'TX',
   'store_zipcode': '78201'},
  {'store_address': '516 S Flores St',
   'store_city': 'San Antonio',
   'store_id': 4,
   'store_state': 'TX',
   'store_zipcode': '78204'},
  {'store_address': '1520 Austin Hwy',
   'store_city': 'San Antonio',
   'store_id': 5,
   'store_state': 'TX',
   'store_zipcode': '78218'},
  {'store_address': '1015 S WW White Rd',
   'store_city': 'San Antonio',
   'store_id': 6,
   'store_state': 'TX',
   'store_zipcode': '78220'},
  {'store_address': '12018 Perrin Beitel 

In [10]:
stores_df = pd.DataFrame(data['payload']['stores'])
stores_df

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218
5,1015 S WW White Rd,San Antonio,6,TX,78220
6,12018 Perrin Beitel Rd,San Antonio,7,TX,78217
7,15000 San Pedro Ave,San Antonio,8,TX,78232
8,735 SW Military Dr,San Antonio,9,TX,78221
9,8503 NW Military Hwy,San Antonio,10,TX,78231


### Creating the 'sales' Data Frame

In [11]:
response = requests.get(base_url + '/api/v1/sales')

In [35]:
data =  response.json()

In [34]:
# This gives us page 2
new_response = requests.get(base_url + data['payload']['next_page'])
new_response.json()

{'payload': {'max_page': 183,
  'next_page': '/api/v1/sales?page=3',
  'page': 2,
  'previous_page': '/api/v1/sales?page=1',
  'sales': [{'item': 1,
    'sale_amount': 33.0,
    'sale_date': 'Sat, 10 Sep 2016 00:00:00 GMT',
    'sale_id': 5001,
    'store': 3},
   {'item': 1,
    'sale_amount': 27.0,
    'sale_date': 'Sun, 11 Sep 2016 00:00:00 GMT',
    'sale_id': 5002,
    'store': 3},
   {'item': 1,
    'sale_amount': 26.0,
    'sale_date': 'Mon, 12 Sep 2016 00:00:00 GMT',
    'sale_id': 5003,
    'store': 3},
   {'item': 1,
    'sale_amount': 22.0,
    'sale_date': 'Tue, 13 Sep 2016 00:00:00 GMT',
    'sale_id': 5004,
    'store': 3},
   {'item': 1,
    'sale_amount': 25.0,
    'sale_date': 'Wed, 14 Sep 2016 00:00:00 GMT',
    'sale_id': 5005,
    'store': 3},
   {'item': 1,
    'sale_amount': 22.0,
    'sale_date': 'Thu, 15 Sep 2016 00:00:00 GMT',
    'sale_id': 5006,
    'store': 3},
   {'item': 1,
    'sale_amount': 35.0,
    'sale_date': 'Fri, 16 Sep 2016 00:00:00 GMT',
    'sal

In [19]:
data['payload']['sales']

[{'item': 1,
  'sale_amount': 13.0,
  'sale_date': 'Tue, 01 Jan 2013 00:00:00 GMT',
  'sale_id': 1,
  'store': 1},
 {'item': 1,
  'sale_amount': 11.0,
  'sale_date': 'Wed, 02 Jan 2013 00:00:00 GMT',
  'sale_id': 2,
  'store': 1},
 {'item': 1,
  'sale_amount': 14.0,
  'sale_date': 'Thu, 03 Jan 2013 00:00:00 GMT',
  'sale_id': 3,
  'store': 1},
 {'item': 1,
  'sale_amount': 13.0,
  'sale_date': 'Fri, 04 Jan 2013 00:00:00 GMT',
  'sale_id': 4,
  'store': 1},
 {'item': 1,
  'sale_amount': 10.0,
  'sale_date': 'Sat, 05 Jan 2013 00:00:00 GMT',
  'sale_id': 5,
  'store': 1},
 {'item': 1,
  'sale_amount': 12.0,
  'sale_date': 'Sun, 06 Jan 2013 00:00:00 GMT',
  'sale_id': 6,
  'store': 1},
 {'item': 1,
  'sale_amount': 10.0,
  'sale_date': 'Mon, 07 Jan 2013 00:00:00 GMT',
  'sale_id': 7,
  'store': 1},
 {'item': 1,
  'sale_amount': 9.0,
  'sale_date': 'Tue, 08 Jan 2013 00:00:00 GMT',
  'sale_id': 8,
  'store': 1},
 {'item': 1,
  'sale_amount': 12.0,
  'sale_date': 'Wed, 09 Jan 2013 00:00:00 GMT

In [32]:
data['payloadload']['next_page']

'/api/v1/sales?page=2'

In [36]:
def get_sales_data():
    url = 'https://python.zgulde.net/api/v1/sales'
    response = requests.get(url)

    filename = 'sales.csv'
    if os.path.isfile(filename):
        sales = pd.read_csv(filename, index_col=[0])
    else:
        if response.ok:
            extracted_data = list()
            payload = response.json()['payload']
            max_page = payload['max_page']
            for n in range(max_page):
                extracted_data.extend(payload['sales'])
                try:
                    new_url = url[:25] + payload['next_page']
                    print(new_url)
                    response = requests.get(new_url)
                    payload = response.json()['payload']
                except:
                    pass
                
            sales = pd.DataFrame(extracted_data)
            sales.to_csv(filename)

        else:
            print(response.status_codeus_code)
    return sales

In [39]:
sales_df = get_sales_data()

In [40]:
sales_df

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1
...,...,...,...,...,...
912995,50,63.0,"Wed, 27 Dec 2017 00:00:00 GMT",912996,10
912996,50,59.0,"Thu, 28 Dec 2017 00:00:00 GMT",912997,10
912997,50,74.0,"Fri, 29 Dec 2017 00:00:00 GMT",912998,10
912998,50,62.0,"Sat, 30 Dec 2017 00:00:00 GMT",912999,10


### Combine the 3

In [46]:
sales_df.head()

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,1
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,1
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,1
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,1


In [47]:
items_df.head()

,level_0,index,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,0,0.0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,1.0,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,2,2.0,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,3,3.0,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,4,4.0,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


In [48]:
stores_df.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


In [52]:
df = sales_df.set_index('store').join(stores_df.set_index('store_id'))

In [59]:
df = df.merge(items_df,how='right',left_on='item',right_on='item_id')

In [60]:
df.shape

(913000, 16)

In [61]:
df.head()

,item,sale_amount,sale_date,sale_id,store_address,store_city,store_state,store_zipcode,level_0,index,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,0,0.0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,11.0,"Wed, 02 Jan 2013 00:00:00 GMT",2,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,0,0.0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
2,1,14.0,"Thu, 03 Jan 2013 00:00:00 GMT",3,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,0,0.0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
3,1,13.0,"Fri, 04 Jan 2013 00:00:00 GMT",4,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,0,0.0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
4,1,10.0,"Sat, 05 Jan 2013 00:00:00 GMT",5,12125 Alamo Ranch Pkwy,San Antonio,TX,78253,0,0.0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [62]:
df.index.value_counts()

0         1
312882    1
290343    1
267816    1
269865    1
         ..
884965    1
891110    1
889063    1
911592    1
2047      1
Length: 913000, dtype: int64

In [63]:
url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
germany_power = pd.read_csv(url)

In [65]:
germany_power

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.18400,NaN,NaN,NaN
1,2006-01-02,1380.52100,NaN,NaN,NaN
2,2006-01-03,1442.53300,NaN,NaN,NaN
3,2006-01-04,1457.21700,NaN,NaN,NaN
4,2006-01-05,1477.13100,NaN,NaN,NaN
...,...,...,...,...,...
4378,2017-12-27,1263.94091,394.507,16.530,411.037
4379,2017-12-28,1299.86398,506.424,14.162,520.586
4380,2017-12-29,1295.08753,584.277,29.854,614.131
4381,2017-12-30,1215.44897,721.247,7.467,728.714
